In [26]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

In [27]:
AAPL_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/AAPL_predictions_SVM.csv", index_col=0)


In [28]:
AAPL_predictions.head()

,Date,Predictions
0,2019-06-28,0
1,2019-05-31,0
2,2019-04-30,1
3,2019-03-29,1
4,2019-02-28,1


In [29]:
AAPL_predictions.Date = pd.to_datetime(AAPL_predictions.Date, format="%Y-%m-%d")

In [30]:
AAPL_predictions['Predictions'] = AAPL_predictions['Predictions'].shift(-1)


In [31]:
AAPL_predictions = AAPL_predictions.dropna()


In [32]:
AAPL_predictions = AAPL_predictions[::-1]

In [33]:
AAPL_predictions.head(10)

,Date,Predictions
111,2010-03-31,1.0
110,2010-04-30,0.0
109,2010-05-28,0.0
108,2010-06-30,1.0
107,2010-07-30,0.0
106,2010-08-31,1.0
105,2010-09-30,1.0
104,2010-10-29,1.0
103,2010-11-30,1.0
102,2010-12-31,1.0


In [34]:
AAPL_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_monthly.csv", index_col =0)

In [35]:
AAPL_monthly.columns = ['Open','High','Low','Close','Volume']

In [36]:
AAPL_predictions = AAPL_predictions.set_index("Date")

In [37]:
AAPL_monthly.head()

,Open,High,Low,Close,Volume
2019-07-26,203.17,209.73,198.41,207.74,349033605
2019-06-28,175.60,201.57,170.27,197.92,515218768
2019-05-31,209.88,215.31,174.99,175.07,739456573
2019-04-30,191.64,208.48,188.38,200.67,506117812
2019-03-29,174.28,197.69,169.50,189.95,650981384


In [38]:
AAPL_final_df = pd.merge(AAPL_predictions, AAPL_monthly, right_index=True, left_index=True)

In [39]:
AAPL_final_df.index = pd.to_datetime(AAPL_final_df.index, format ="%Y-%m-%d")

In [40]:
equity_dict = dict()

In [41]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0  and (position.is_long != True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0  and (position.is_short !=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [42]:
bt = Backtest(AAPL_final_df, Binary_Classification_1d, cash=10000, commission=.02)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-06-28 00:00:00
Duration                        3376 days 00:00:00
Exposure [%]                               94.7275
Equity Final [$]                            659758
Equity Peak [$]                             668925
Return [%]                                 6497.58
Buy & Hold Return [%]                      15.7787
Max. Drawdown [%]                         -12.7573
Avg. Drawdown [%]                         -4.13669
Max. Drawdown Duration           245 days 00:00:00
Avg. Drawdown Duration            87 days 00:00:00
# Trades                                        45
Win Rate [%]                               77.7778
Best Trade [%]                             88.3194
Worst Trade [%]                           -6.54411
Avg. Trade [%]                             10.1262
Max. Trade Duration              332 days 00:00:00
Avg. Trade Duration               72 days 00:00:00
Expectancy [%]                 

In [43]:
bt.plot(superimpose=False, plot_volume=False)

In [44]:
df = pd.DataFrame.from_dict([equity_dict])

In [45]:
df = df.T

In [46]:
df.columns = ['Equity']

In [47]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/AAPL_SVM_Equity.csv")

In [48]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,10062.545333
2010-06-30,10261.343773
2010-07-30,10074.761656
2010-08-31,10665.474925
2010-09-30,11800.672574
2010-10-29,12517.238524
2010-11-30,12940.191264
2010-12-31,13414.713464
2011-01-31,14111.732926
